In [21]:
#Netflows are total count of occurence of a src_ip in a particular window
import pandas as pd
import numpy as np
#Edit path to the file for extracting features
def getFeatures(file_path):
    df=pd.read_csv(file_path)
    temp = pd.get_dummies(df.Tag)
    df['attack'] = temp.Attack
    df['normal'] = temp.Normal
    del temp, df['Tag']
    group = df.groupby(['bucket','source'])
    features = group.sourcePort.nunique()
    features = pd.DataFrame(features)
    features.rename(columns={'sourcePort':'usrc_port'},inplace=True)
    features['udest_ip'] = group.destination.nunique()
    features['udest_port'] = group.destinationPort.nunique()
    features['netflows'] = group.destination.count()
    features['bytes'] = group.totalSourceBytes.sum()
    features['packets'] = group.totalSourcePackets.sum()
    features['attack'] = group.attack.sum()
    features['normal'] = group.normal.sum()
    features.reset_index(inplace=True) 
    return features
    
def featurestoArray(features):
    #Converting DF to np array
    features_array = features.copy()
    del features_array['attack'],features_array['normal'],features_array['source'],features_array['bucket']
    features_array = features_array.as_matrix()
    return features_array

def getClusterFeatures(labeled_features):
    group = labeled_features.groupby(['label'])
    clusterfeatures = group.bucket.count()
    clusterfeatures = pd.DataFrame(clusterfeatures)
    clusterfeatures.rename(columns={'bucket':'instances'},inplace=True) 
    clusterfeatures['netflows']=group.netflows.sum()
    clusterfeatures['avgnetflows']=group.netflows.mean()
    clusterfeatures['stdnetflows']=group.netflows.std()
    clusterfeatures['usrc_ip']=group.source.nunique()
    clusterfeatures['avgsrc_port']=group.usrc_port.mean()
    clusterfeatures['stdsrc_port']=group.usrc_port.std()
    clusterfeatures['avgdest_ip']=group.udest_ip.mean()
    clusterfeatures['stddest_ip']=group.udest_ip.std()
    clusterfeatures['avgdest_port']=group.udest_port.mean()
    clusterfeatures['stddest_port']=group.udest_port.std()
    clusterfeatures['avgbytes']=group.bytes.mean()
    clusterfeatures['stdbytes']=group.bytes.std()
    clusterfeatures['avgpackets']=group.packets.mean()
    clusterfeatures['stdpackets']=group.packets.std()
    clusterfeatures['attack']=group.attack.sum()
    clusterfeatures['normal']=group.normal.sum()
    #True = Botnet    False = Normal
    clusterfeatures['label'] = (group.attack.sum()/group.normal.sum()>0.01)
    return clusterfeatures

In [22]:
def getLabeledFeaturesGMM(features_array):
    X_train = features_array
    from sklearn import mixture
    #Number of clusters = number of components
    # Four covariance Type ['spherical', 'diag', 'tied', 'full'] iterations can be changed
    clf = mixture.GMM(n_components=40, covariance_type='full', n_iter=100)
    label = clf.fit_predict(X_train)
    return label           

def addLabelTofeatures(features, label):
    labeled_features = features.copy()
    labeled_features['label'] = label
    return labeled_features

In [31]:
#features = getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/13Junflow_ndf.csv")
#features = pd.concat([features,getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/14junflow_ndf.csv")],axis=0)
features = pd.concat([features,getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/15junflow1_ndf.csv")],axis=0) 
features = pd.concat([features,getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/15junflow2_ndf.csv")],axis=0) 
features = pd.concat([features,getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/15junflow3_ndf.csv")],axis=0) 
#features = pd.concat([features,getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/16junflow1_ndf.csv")],axis=0) 
features = pd.concat([features,getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/16junflow2_ndf.csv")],axis=0) 
features = pd.concat([features,getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/16junflow3_ndf.csv")],axis=0)
features = pd.concat([features,getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/17junflow3_ndf.csv")],axis=0) 


In [32]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import Imputer 
X_train, X_test = train_test_split(features, test_size=0.3,random_state=0)
X_train

,bucket,source,usrc_port,udest_ip,udest_port,netflows,bytes,packets,attack,normal
1067,65.0,192.168.4.119,1,1,1,1,531.0,8.0,0.0,1.0
10732,734.0,192.168.2.109,4,5,2,5,1166.0,7.0,0.0,5.0
17000,601.0,222.165.223.2,1,1,1,3,56.0,0.0,0.0,3.0
4484,220.0,192.168.2.112,20,8,1,21,9224.0,116.0,0.0,21.0
6613,338.0,192.168.1.104,1,1,1,1,515.0,7.0,0.0,1.0
378,30.0,192.168.1.102,26,3,1,30,45263.0,500.0,0.0,30.0
6604,338.0,187.58.152.198,1,1,1,1,55.0,0.0,0.0,1.0
6766,340.0,62.105.24.211,1,1,1,1,4.0,0.0,0.0,1.0
1045,67.0,192.168.1.104,4,4,2,4,778.0,9.0,0.0,4.0
952,52.0,192.168.1.104,4,2,1,4,3399.0,45.0,0.0,4.0


In [33]:
feature_array_train = featurestoArray(features=X_train)
train_label = getLabeledFeaturesGMM(features_array=feature_array_train)
X_train = addLabelTofeatures(features=X_train,label=train_label)
train_cluster_features = getClusterFeatures(labeled_features=X_train) 
train_labels = train_cluster_features['label']
del train_cluster_features['attack'],train_cluster_features['normal'],train_cluster_features['label']
train_cluster_features = Imputer.fit_transform(Imputer(strategy="most_frequent",axis=0),train_cluster_features)

feature_array_test = featurestoArray(features=X_test)
test_label = getLabeledFeaturesGMM(features_array=feature_array_test)
X_test = addLabelTofeatures(features=X_test,label=test_label)
test_cluster_features = getClusterFeatures(labeled_features=X_test)
test_labels = test_cluster_features['label']
del test_cluster_features['attack'],test_cluster_features['normal'],test_cluster_features['label'] 
#test_cluster_features = Imputer.fit_transform(Imputer(strategy="most_frequent",axis=0),test_cluster_features)

/home/ankit/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:52: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/ankit/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The functon distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/home/ankit/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/home/ankit/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:

In [34]:
train_cluster_features = train_cluster_features.astype(np.float32)
train_labels = train_labels.astype(np.float32)

test_cluster_features = test_cluster_features.astype(np.float32)
test_labels = test_labels.astype(np.float32)
#Using Gaussian Naive Bayes Classifier for predicting cluster labels since our clustering is also Gaussian
import sklearn.preprocessing
train_cluster_features= sklearn.preprocessing.normalize(train_cluster_features,axis=0)
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(feature_array_train, train_label)
cluster_label_predict = gnb.predict(feature_array_test)
cluster_label_predict

array([22, 13, 21, ..., 36, 22,  0])

In [35]:
#Confusion Matrix of flows 
label_predict = [train_labels[x[0]] for x in zip(cluster_label_predict)]
label_actual = [test_labels[x[0]] for x in  zip(test_label)]

from sklearn.metrics import confusion_matrix
confusion_matrix(label_actual, label_predict) #0 = normal

array([[45240,   127],
       [  317,   566]])

In [36]:
X_test['pred'] = label_predict
X_test['act_label'] = label_actual
X_test.source.unique()

array(['192.168.1.101', '192.168.5.122', '192.168.4.120', ...,
       '178.49.253.44', '80.99.94.56', '90.191.161.122'], dtype=object)

In [37]:
#Predicted Malicious IP's
malicious_IP = X_test.groupby(['pred']).get_group(1)['source']
malicious_IP.unique()

array(['192.168.4.120', '192.168.4.118', '192.168.2.109', '192.168.2.113',
       '192.168.2.110', '192.168.1.103', '192.168.1.105', '192.168.5.123',
       '192.168.1.101', '192.168.3.115', '192.168.4.121', '65.55.34.12',
       '131.202.243.90', '192.168.3.117', '192.168.3.114', '192.168.2.107',
       '192.168.1.102'], dtype=object)

In [38]:
#Actual Malicious IP's 
malicious_act = X_test.groupby(['act_label']).get_group(1)['source']
malicious_act.unique()

array(['192.168.1.101', '192.168.4.120', '192.168.2.112', '192.168.4.118',
       '192.168.2.113', '192.168.2.110', '192.168.1.103', '192.168.2.109',
       '192.168.1.105', '192.168.4.119', '192.168.5.123', '192.168.3.115',
       '192.168.2.111', '192.168.3.116', '192.168.4.121', '192.168.3.114',
       '192.168.1.104', '65.55.34.12', '192.168.2.107', '192.168.1.102',
       '192.168.2.108', '192.168.2.106', '192.168.3.117', '131.202.243.90'], dtype=object)